In [1]:
import numpy as np
from bosonicplus.base import State
from bosonicplus.operations.symplectic import squeezing

In [2]:
#Check that photon-number moment calculation works on a squeezed state
state = State(1)
r = 1
state.apply_symplectic(squeezing(r,0 ))

print(state.get_photon_number_moments())
print('nbar', np.sinh(r)**2)
print('nvar', 2 * np.sinh(r)**2*np.cosh(r)**2)

(1.3810978455418155, 6.57705820900412)
nbar 1.3810978455418155
nvar 6.577058209004121


In [3]:
#Photon number moment of a coherent state
state = State(1)
alpha = 0.5
state.apply_displacement(np.array([alpha*np.sqrt(2*state.hbar),0]))
print(state.get_photon_number_moments())
print(alpha**2)

(0.25, 0.24999999999999994)
0.25


In [4]:
#Calculating the photon-number moments in the coherent state decomposition

from bosonicplus.states.nongauss import prepare_fock_coherent

#Fock state 
ns = np.arange(10)
for n in ns:
    fock = prepare_fock_coherent(n, inf = 1e-10, fast = True) 
    ex, var = fock.get_photon_number_moments()
    print(ex.real, var.real)

1.0000000827403705e-10 9.999999999999996e-11
1.0000000001991225 1.0000000001987743
2.0000000000316818 2.000000000037667
3.000000000912946 3.000000000895086
4.000000000441295 4.000000000441288
5.000000000490432 5.000000000490432
6.000000000923478 6.000000000923478
7.0000000008946826 7.0000000008946826
8.000000000937657 8.000000000937657
9.000000000937037 9.000000000937037


In [5]:
#Apply Gaussian ops on a Fock state

n = 3
eta = 1
r = 1
state = prepare_fock_coherent(n, inf = 1e-5, fast = True) 
state.apply_loss(np.array([eta]), np.zeros(1))
state.apply_symplectic(squeezing(r,0))
ex, var = state.get_photon_number_moments()
print(ex)
print(var)

(12.667835405434724-52.90988098408059j)
(88.50284903776038-206.14065659279095j)


In [6]:
#Check with strawberryfields
import strawberryfields as sf
from strawberryfields.ops import Sgate, BSgate, MeasureFock, MeasureHomodyne, Fock, Catstate, LossChannel

In [7]:
eng = sf.Engine('fock', backend_options={"cutoff_dim": 200})
prog = sf.Program(1)


with prog.context as q:
    Fock(n) | q[0]
    LossChannel(eta) | q[0]
    Sgate(r,0) | q[0]
    
result = eng.run(prog)

state = result.state
state.mean_photon(0)

(12.667684918792709, 85.50175671705358)